In [6]:
import pandas as pd
from hyperopt import hp

from pyprophet.io.scoring.osw import OSWReader
from pyprophet.io.scoring.parquet import ParquetReader
from pyprophet.io.scoring.split_parquet import SplitParquetReader
from pyprophet.io.scoring.tsv import TSVReader  # legacy, limited support
from pyprophet._config import RunnerIOConfig, RunnerConfig, IPFIOConfig

In [37]:
d = pd.read_parquet("./data/test_data.oswpq/transition_features.parquet")
# (96,259, 23)
d

,RUN_ID,IPF_PEPTIDE_ID,PRECURSOR_ID,TRANSITION_ID,TRANSITION_TRAML_ID,PRODUCT_MZ,TRANSITION_CHARGE,TRANSITION_TYPE,TRANSITION_ORDINAL,ANNOTATION,...,FEATURE_TRANSITION_VAR_LOG_SN_SCORE,FEATURE_TRANSITION_VAR_MASSDEV_SCORE,FEATURE_TRANSITION_VAR_ISOTOPE_CORRELATION_SCORE,FEATURE_TRANSITION_VAR_ISOTOPE_OVERLAP_SCORE,IPF_PEPTIDE_ID_1,SCORE_TRANSITION_SCORE,SCORE_TRANSITION_RANK,SCORE_TRANSITION_P_VALUE,SCORE_TRANSITION_Q_VALUE,SCORE_TRANSITION_PEP
0,-8.670811e+18,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-8.670811e+18,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-8.670811e+18,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-8.670811e+18,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-8.670811e+18,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96254,-8.670811e+18,140.0,267.0,17803.0,10732_UISDECOY_{CCAHELNK(Label:13C(6)15N(2))LI...,771.2913,1.0,b,7.0,b7^1,...,0.626884,5.69439,-0.353037,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
96255,-8.670811e+18,140.0,267.0,17821.0,10758_UISDECOY_{CCAHELNK(Label:13C(6)15N(2))LI...,865.5597,1.0,y,7.0,y7^1,...,3.270770,3.27601,0.695822,2.366670,NaN,NaN,NaN,NaN,NaN,NaN
96256,-8.670811e+18,140.0,267.0,17823.0,10760_UISDECOY_{CCAHELNK(Label:13C(6)15N(2))LI...,1002.6186,1.0,y,8.0,y8^1,...,1.047340,15.04500,0.727999,0.666667,NaN,NaN,NaN,NaN,NaN,NaN
96257,-8.670811e+18,140.0,267.0,17825.0,10762_UISDECOY_{CCAHELNK(Label:13C(6)15N(2))LI...,1073.6557,1.0,y,9.0,y9^1,...,2.740000,10.40250,0.841894,0.500000,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
d1 = d[ (~d[['TRANSITION_ID']].isna().values) & (~d[['FEATURE_ID']].isna().values) & (~d[['SCORE_TRANSITION_SCORE']].isna().values) ]
sore_df = d1[['IPF_PEPTIDE_ID', 'PRECURSOR_ID', 'TRANSITION_ID', 'FEATURE_ID', 'SCORE_TRANSITION_SCORE']].drop_duplicates().sort_values(by=['TRANSITION_ID', 'FEATURE_ID'])

In [36]:
pd.merge(d, sore_df, how='left', on=['IPF_PEPTIDE_ID', 'TRANSITION_ID', 'FEATURE_ID'])

,RUN_ID,IPF_PEPTIDE_ID,PRECURSOR_ID_x,TRANSITION_ID,TRANSITION_TRAML_ID,PRODUCT_MZ,TRANSITION_CHARGE,TRANSITION_TYPE,TRANSITION_ORDINAL,ANNOTATION,...,FEATURE_TRANSITION_APEX_INTENSITY,FEATURE_TRANSITION_VAR_LOG_INTENSITY,FEATURE_TRANSITION_VAR_XCORR_COELUTION,FEATURE_TRANSITION_VAR_XCORR_SHAPE,FEATURE_TRANSITION_VAR_LOG_SN_SCORE,FEATURE_TRANSITION_VAR_MASSDEV_SCORE,FEATURE_TRANSITION_VAR_ISOTOPE_CORRELATION_SCORE,FEATURE_TRANSITION_VAR_ISOTOPE_OVERLAP_SCORE,PRECURSOR_ID_y,SCORE_TRANSITION_SCORE
0,-8.670811e+18,522.0,119.0,4723.0,5376_UIS_{GIGDWSDSK(Label:13C(6)15N(2))}_486.7...,357.1860,1.0,y,3.0,y3^1,...,5439.0,10.42130,0.00,0.956729,1.126040,4.63905,0.892007,0.206288,119.0,1.888922
1,-8.670811e+18,522.0,119.0,4720.0,5369_UIS_{GIGDWSDSK(Label:13C(6)15N(2))}_486.7...,366.1534,2.0,b,7.0,b7^2,...,1121.0,9.74062,3.25,0.642226,0.616837,32.70270,0.590331,0.299732,119.0,-1.541282
2,-8.670811e+18,522.0,119.0,4727.0,5383_UIS_{GIGDWSDSK(Label:13C(6)15N(2))}_486.7...,373.1658,2.0,y,6.0,y6^2,...,1050.0,9.48403,2.75,0.439791,0.000000,6.50160,0.516150,1.649520,NaN,NaN
3,-8.670811e+18,522.0,119.0,4729.0,5385_UIS_{GIGDWSDSK(Label:13C(6)15N(2))}_486.7...,401.6765,2.0,y,7.0,y7^2,...,702.0,8.69484,0.75,0.756959,2.143890,6.46771,0.457555,1.542740,NaN,NaN
4,-8.670811e+18,522.0,119.0,4722.0,5371_UIS_{GIGDWSDSK(Label:13C(6)15N(2))}_486.7...,409.6694,2.0,b,8.0,b8^2,...,60.0,6.52356,4.25,0.348838,0.000000,17.22790,-0.166539,11.033300,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96254,-8.670811e+18,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96255,-8.670811e+18,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96256,-8.670811e+18,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96257,-8.670811e+18,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
xgb_hyperparams = {
    "autotune": False,
    "autotune_num_rounds": 10,
    "num_boost_round": 100,
    "early_stopping_rounds": 10,
    "test_size": 0.33,
}

xgb_params = {
    "eta": 0.3,
    "gamma": 0,
    "max_depth": 6,
    "min_child_weight": 1,
    "subsample": 1,
    "colsample_bytree": 1,
    "colsample_bylevel": 1,
    "colsample_bynode": 1,
    "lambda": 1,
    "alpha": 0,
    "scale_pos_weight": 1,
    "verbosity": 0,
    "objective": "binary:logitraw",
    "nthread": 1,
    "eval_metric": "auc",
}

xgb_params_space = {
    "eta": hp.uniform("eta", 0.0, 0.3),
    "gamma": hp.uniform("gamma", 0.0, 0.5),
    "max_depth": hp.quniform("max_depth", 2, 8, 1),
    "min_child_weight": hp.quniform("min_child_weight", 1, 5, 1),
    "lambda": hp.uniform("lambda", 0.0, 1.0),
    "alpha": hp.uniform("alpha", 0.0, 1.0),
    "objective": "binary:logitraw",
    "nthread": 1,
    "eval_metric": "auc",
    "scale_pos_weight": 1.0,
    "verbosity": 0,
}

def create_reader_config(level, infile, outfile):
    """
    Common config generator to avoid repetition
    """
    return RunnerIOConfig(
        infile=infile,
        outfile=outfile,
        subsample_ratio=1,
        context="score_learn",
        level=level,
        runner=RunnerConfig(
            xgb_hyperparams=xgb_hyperparams,
            xgb_params=xgb_params,
            xgb_params_space=xgb_params_space,
        ),
    )
    
def _create_osw_reader(level):
    config = create_reader_config(
        level, "./data/test_data.osw", "./data/tmp_test_data.osw"
    )
    return OSWReader(config)


def _create_parquet_reader(level):
    config = create_reader_config(
        level, "./data/test_data.parquet", "./data/tmp_test_data.parquet"
    )
    return ParquetReader(config)


def _create_split_parquet_reader(level):
    config = create_reader_config(
        level, "./data/test_data.oswpq/", "./data/tmp_test_data.oswpq/"
    )
    return SplitParquetReader(config)


def _create_split_parquet_multi_reader(level):
    config = create_reader_config(
        level, "./data/test_data.oswpqd/", "./data/tmp_test_data.oswpqd/"
    )
    return SplitParquetReader(config)

In [3]:
osw_ms1 = _create_osw_reader("ms1").read()
parquet_ms1 = _create_parquet_reader("ms1").read()

Warn: SQLite index creation failed: no such table: main.SCORE_MS2


In [4]:
osw_ms1

,feature_id,area_intensity,apex_intensity,var_massdev_score,var_isotope_correlation_score,var_isotope_overlap_score,var_xcorr_coelution,main_var_xcorr_shape,run_id,precursor_id,exp_rt,precursor_charge,decoy,group_id
0,260819276075322832,104006.0,8790.781242,5.420215,0.955497,0.266667,5.142969,0.653220,-8670811102654834151,0,2605.74,2,0,-8670811102654834151_0
1,-4409520928686189639,854645.0,117220.748195,1.370667,0.983495,0.124750,0.000000,0.990675,-8670811102654834151,0,2661.55,2,0,-8670811102654834151_0
2,8534214264242363560,73215.2,5750.471592,7.161017,-0.369161,0.749815,6.750000,0.482709,-8670811102654834151,0,2708.53,2,0,-8670811102654834151_0
3,6932937885234622359,25862.3,4036.559980,1.850493,0.187202,2.443450,4.008306,0.642192,-8670811102654834151,0,2795.06,2,0,-8670811102654834151_0
4,5163914660633416481,241873.0,10419.743453,8.990706,0.612257,0.470680,4.008306,0.598525,-8670811102654834151,0,2832.77,2,0,-8670811102654834151_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3405,9031575518883827227,11974.7,636.935673,0.761631,-0.093248,1.250000,6.612908,0.481051,-8670811102654834151,683,3142.26,2,1,-8670811102654834151_683
3406,-5247509123739442138,14790.3,1171.596045,3.174701,-0.584060,1.166580,3.790994,0.511116,-8670811102654834151,683,3452.76,2,1,-8670811102654834151_683
3407,-860969257393568588,60859.5,7442.936204,1.047568,0.298453,1.840175,5.609126,0.490877,-8670811102654834151,683,3498.66,2,1,-8670811102654834151_683
3408,1493676385987588447,109029.0,16829.052734,8.425311,0.327351,0.153834,2.790994,0.490355,-8670811102654834151,683,3574.66,2,1,-8670811102654834151_683


In [5]:
parquet_ms1

,run_id,precursor_id,precursor_charge,feature_id,exp_rt,decoy,area_intensity,apex_intensity,var_massdev_score,var_isotope_correlation_score,var_isotope_overlap_score,var_xcorr_coelution,main_var_xcorr_shape,group_id
0,-8670811102654834151,0,2,260819276075322832,2605.74,False,104006.000000,8790.781250,5.420215,0.955497,0.266667,5.142970,0.653220,-8670811102654834151_0
1,-8670811102654834151,0,2,-4409520928686189639,2661.55,False,854645.000000,117220.750000,1.370667,0.983495,0.124750,0.000000,0.990675,-8670811102654834151_0
2,-8670811102654834151,0,2,8534214264242363560,2708.53,False,73215.203125,5750.471680,7.161017,-0.369161,0.749815,6.750000,0.482709,-8670811102654834151_0
3,-8670811102654834151,0,2,6932937885234622359,2795.06,False,25862.300781,4036.560059,1.850493,0.187202,2.443450,4.008306,0.642192,-8670811102654834151_0
4,-8670811102654834151,0,2,5163914660633416481,2832.77,False,241873.000000,10419.743164,8.990706,0.612257,0.470680,4.008306,0.598525,-8670811102654834151_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3405,-8670811102654834151,683,2,9031575518883827227,3142.26,True,11974.700195,636.935669,0.761631,-0.093248,1.250000,6.612908,0.481051,-8670811102654834151_683
3406,-8670811102654834151,683,2,-5247509123739442138,3452.76,True,14790.299805,1171.596069,3.174701,-0.584060,1.166580,3.790994,0.511116,-8670811102654834151_683
3407,-8670811102654834151,683,2,-860969257393568588,3498.66,True,60859.500000,7442.936035,1.047568,0.298453,1.840175,5.609127,0.490877,-8670811102654834151_683
3408,-8670811102654834151,683,2,1493676385987588447,3574.66,True,109029.000000,16829.052734,8.425311,0.327351,0.153834,2.790994,0.490355,-8670811102654834151_683
